In [9]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import string
import time
import os


In [2]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()

'C:\\Users\\H P\\Envs\\scrapperenv\\lib\\site-packages\\chromedriver_autoinstaller\\105\\chromedriver.exe'

In [3]:
driver  = webdriver.Chrome(options=opt)

In [54]:
chars   = string.ascii_lowercase

In [50]:
skipped  = []
erroneous= {}

for char in chars:
    data = []
    url  = f'https://nvbar.org/for-the-public/find-a-lawyer/?usearch={char}' 
    count = 0
    while count < 3:
        try:
            driver.get (url)
            time.sleep(3)
            break
        except:count +=1
    
    if count == 3:
        print("skipping ",  char); skipped.append(char)
    
    else:
        articles = driver.find_elements_by_class_name('user_chunk ')
        for c, article in enumerate( articles ):
            item = {} 
            divs = article.find_elements_by_class_name("one_third")
            if len(divs) == 3:
                div1 = divs[0]
                div2 = divs[1]
                div3 = divs[2]

                item['Name'] =  div1.find_element_by_tag_name("h3").text
                for li in div1.find_elements_by_tag_name("li"):
                    key, val = li.text.split(":")
                    item[key.strip()] = val.strip() 

                for i in div2.text.split("\n"):
                    try:
                        key, val         = i.split(":")
                        item[key.strip()]= val.strip()

                    except:
                        item['Address'] = i

                for i in div3.text.split("\n"):
                    try:
                        key, val = i.split(":")
                        item[key.strip()]= val.strip()
                    except:pass 

                item = {key:item[key] if key in item.keys() else "" for key in ('Specialization', 'Name', 'Bar #', 'Member since', 'Status', 'Company', 'Address', 'Phone', 'Fax', 'Email', 'Law school')}
                data.append( item )
            else:
                if char in erroneous.keys():
                    erroneous[char].append(divs)
                else:
                    erroneous[char] = [divs]
                    
            print(f"[{char}] {c}/{len(articles)}", end="\r")
        
        if len(data) > 0:
            df = pd.DataFrame(data)
            df.to_excel(f"files/{char}_data.xlsx", index=False)
        
        else:
            print(f"no record available for {char} - ignoring")
            skipped.append(char)

C:\Users\H P\Envs\scrapperenv\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")
C:\Users\H P\Envs\scrapperenv\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")
C:\Users\H P\Envs\scrapperenv\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
